In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Conv2D, UpSampling2D, Lambda
from tensorflow.keras.models import Model
import tensorflow_hub as hub
from tensorflow.keras.layers import Layer
import numpy as np
from tensorflow.keras import mixed_precision
from tensorflow.keras.layers import Dense
import os
from skimage import io


# Enable mixed precision training
#policy = mixed_precision.Policy("mixed_float16")
#mixed_precision.set_global_policy(policy)

In [ ]:
def load_and_split_image(image_path, img_size=(192, 288)):
    full_image = io.imread(image_path)  
    bscan = full_image[:, :570, :]
    mask = full_image[:, 570:, :]
    bscan = tf.image.resize(bscan, img_size)
    mask = tf.image.resize(mask, img_size)
    return bscan.numpy(), mask.numpy()

COLOR_MAP = {
    (255, 0, 0): 0, (255, 192, 203): 1, (0, 0, 255): 2,
    (0, 255, 0): 3, (255, 255, 0): 4, (0, 0, 0): 5
}

def rgb_to_label(mask_rgb):
    label_mask = np.zeros(mask_rgb.shape[:2], dtype=np.uint8)
    for rgb, class_idx in COLOR_MAP.items():
        matches = np.all(mask_rgb == np.array(rgb), axis=-1)
        label_mask[matches] = class_idx
    return label_mask
    

In [ ]:
# Ensure the conversion of RGB to label mask happens correctly
def load_dataset_from_txt(txt_path, base_dir, img_size=(192, 288)):
    with open(txt_path, "r") as f:
        filenames = [line.strip() for line in f.readlines()]
    images, masks = [], []
    for filename in filenames:
        eye_id = filename.split("_")[0]
        image_path = os.path.join(base_dir, eye_id, filename)
        bscan, mask_rgb = load_and_split_image(image_path, img_size)
        mask_label = rgb_to_label(mask_rgb)  # Convert RGB mask to label format
        images.append(bscan)
        masks.append(mask_label)
    return np.array(images), np.array(masks)

In [ ]:
class GraphAttentionLayer(Layer):
    def __init__(self, units, **kwargs):
        super(GraphAttentionLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        feature_dim = input_shape[0][-1]  # Extract last dimension (features per node)
        self.kernel = self.add_weight(name="kernel", shape=(feature_dim, self.units), initializer="glorot_uniform")
        self.att_kernel = self.add_weight(name="att_kernel", shape=(self.units, self.units), initializer="glorot_uniform")

    def call(self, inputs):
        node_features, adj_matrix = inputs  
        adj_matrix = tf.convert_to_tensor(adj_matrix, dtype=tf.float32)

        # Transform node features (Linear transformation)
        transformed_features = tf.matmul(node_features, self.kernel)

        # Compute attention scores (Now ensuring valid dimensions)
        attention_logits = tf.matmul(transformed_features, self.att_kernel)
        attention_scores = tf.nn.softmax(attention_logits, axis=1)

        # **Fix: Element-wise multiplication instead of tf.matmul**
        weighted_sum = attention_scores * transformed_features  # Element-wise multiply
        return weighted_sum

# Fix adjacency matrix creation
def create_adjacency_matrix(height, width):
    N = height * width
    adj_matrix = np.zeros((N, N))
    for i in range(height):
        for j in range(width):
            node_idx = i * width + j
            if i > 0:
                adj_matrix[node_idx, (i - 1) * width + j] = 1  # Top neighbor
            if i < height - 1:
                adj_matrix[node_idx, (i + 1) * width + j] = 1  # Bottom neighbor
            if j > 0:
                adj_matrix[node_idx, i * width + (j - 1)] = 1  # Left neighbor
            if j < width - 1:
                adj_matrix[node_idx, i * width + (j + 1)] = 1  # Right neighbor
    return tf.convert_to_tensor(adj_matrix, dtype=tf.float32)


In [ ]:
def unet_vit_gat_model(input_size=(64, 64, 3), num_classes=6):  # Reduced input size for efficiency
    inputs = Input(input_size)
    H, W, C = input_size

    # Reshape image into a feature matrix (N, F)
    reshaped = Reshape((H * W, C))(inputs)

    # Generate adjacency matrix once
    adj_matrix = create_adjacency_matrix(H, W)

    # Apply GAT
    x = GraphAttentionLayer(units=128)([reshaped, adj_matrix])

    # Reshape back to image (H, W, F)
    x = Reshape((H, W, 128))(x)

    # U-Net style decoder with a single upsampling to yield 128x128 output
    x = UpSampling2D((2, 2))(x)  # Upsample from 64x64 to 128x128
    x = Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    
    # Output layer: now the model outputs a segmentation mask of shape (128, 128, num_classes)
    outputs = Conv2D(num_classes, (1, 1), activation="softmax", padding="same")(x)

    return Model(inputs, outputs, name="UNet_ViT_GAT")


In [ ]:
# Load dataset (your load_dataset_from_txt function remains unchanged)
X_train, Y_train = load_dataset_from_txt("/kaggle/input/amd-sd/AMD-SD/training.txt", 
                                           "/kaggle/input/amd-sd/AMD-SD/images")
X_val, Y_val = load_dataset_from_txt("/kaggle/input/amd-sd/AMD-SD/validation.txt", 
                                       "/kaggle/input/amd-sd/AMD-SD/images")

# Normalize input images
X_train = X_train.astype("float32") / 255.0
X_val = X_val.astype("float32") / 255.0

X_train = tf.image.resize(X_train, (64, 64))
X_val = tf.image.resize(X_val, (64, 64))

# Convert labels to tensor and add a channel dimension: (samples, height, width, 1)
Y_train = tf.convert_to_tensor(Y_train, dtype=tf.int32)[..., None]
Y_val = tf.convert_to_tensor(Y_val, dtype=tf.int32)[..., None]

# Resize labels to (128, 128) using nearest neighbor to preserve discrete values
Y_train_resized = tf.image.resize(Y_train, (128, 128), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
Y_val_resized = tf.image.resize(Y_val, (128, 128), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

# Remove the extra channel (squeeze) so that shape becomes (samples, 128, 128)
Y_train_squeezed = tf.squeeze(Y_train_resized, axis=-1)
Y_val_squeezed = tf.squeeze(Y_val_resized, axis=-1)

# One-hot encode the labels with num_classes = 6. 
# This converts shape (samples, 128, 128) to (samples, 128, 128, 6)
num_classes = 6
Y_train_one_hot = tf.keras.utils.to_categorical(Y_train_squeezed, num_classes=num_classes)
Y_val_one_hot = tf.keras.utils.to_categorical(Y_val_squeezed, num_classes=num_classes)

# Verify final shapes:
print("X_train shape:", X_train.shape)              # Expect (2346, 64, 64, 3)
print("Y_train_one_hot shape:", Y_train_one_hot.shape)  # Expect (2346, 128, 128, 6)
print("X_val shape:", X_val.shape)
print("Y_val_one_hot shape:", Y_val_one_hot.shape)


In [ ]:
import glob

# Instantiate and inspect the model
model = unet_vit_gat_model()
print("Model output shape:", model.output_shape)  # Should be (None, 128, 128, 6)
model.summary()

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

checkpoint_dir = "/kaggle/working/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Define checkpoint callback (use `.keras` instead of `.h5`)
checkpoint_path = os.path.join(checkpoint_dir, "model_epoch_{epoch:02d}.keras")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,  # Save full model (architecture + weights)
    save_best_only=False,  # Save model at every epoch
    verbose=1
)

# Find the latest saved model
model_files = sorted(glob.glob("/kaggle/working/checkpoints/model_epoch_*.keras"))

if model_files:
    last_checkpoint = model_files[-1]  # Get the last saved model
    print("🔄 Resuming training from:", last_checkpoint)
    model = tf.keras.models.load_model(last_checkpoint)  # Load model
    initial_epoch = int(last_checkpoint.split("_")[-1].split(".keras")[0])  # Extract epoch number
else:
    print("🚀 No saved model found, starting from scratch.")
    initial_epoch = 0  # Start from epoch 0

# Train the model using the processed data
history = model.fit(
    X_train, Y_train_one_hot, epochs=10, batch_size=4, 
    validation_data=(X_val, Y_val_one_hot),
    initial_epoch=initial_epoch,  # Use the correct starting epoch
    callbacks=[checkpoint_callback]  # Make sure to include the checkpoint callback
)
# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, Y_val_one_hot)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


In [ ]:
# import matplotlib.pyplot as plt

# # Assuming `history` is the output of model.fit()
# def plot_training_history(history):
#     epochs = range(1, len(history.history["loss"]) + 1)

#     # Plot Loss
#     plt.figure(figsize=(12, 5))
    
#     plt.subplot(1, 2, 1)
#     plt.plot(epochs, history.history["loss"], label="Training Loss")
#     plt.plot(epochs, history.history["val_loss"], label="Validation Loss")
#     plt.xlabel("Epochs")
#     plt.ylabel("Loss")
#     plt.title("Training & Validation Loss")
#     plt.legend()

#     # Plot Accuracy
#     plt.subplot(1, 2, 2)
#     plt.plot(epochs, history.history["accuracy"], label="Training Accuracy")
#     plt.plot(epochs, history.history["val_accuracy"], label="Validation Accuracy")
#     plt.xlabel("Epochs")
#     plt.ylabel("Accuracy")
#     plt.title("Training & Validation Accuracy")
#     plt.legend()

#     plt.show()

# # Call the function
# plot_training_history(history)


In [ ]:
# import numpy as np
# import tensorflow as tf
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
# from scipy.spatial.distance import directed_hausdorff
# import pandas as pd
# from IPython.display import display

# # Function to compute Dice score
# def dice_score(y_true, y_pred):
#     smooth = 1e-6  # To avoid division by zero
#     intersection = np.sum(y_true * y_pred)
#     dice = (2.0 * intersection + smooth) / (np.sum(y_true) + np.sum(y_pred) + smooth)
#     return dice

# # Function to compute IoU (Jaccard Index)
# def iou_score(y_true, y_pred):
#     smooth = 1e-6
#     intersection = np.sum(y_true * y_pred)
#     union = np.sum(y_true) + np.sum(y_pred) - intersection
#     return (intersection + smooth) / (union + smooth)

# # Function to compute Hausdorff Distance
# def hausdorff_distance(y_true, y_pred):
#     y_true_points = np.argwhere(y_true > 0)
#     y_pred_points = np.argwhere(y_pred > 0)
    
#     if y_true_points.shape[0] == 0 or y_pred_points.shape[0] == 0:
#         return np.nan  # Return NaN if no points are present (empty segmentation)

#     d1 = directed_hausdorff(y_true_points, y_pred_points)[0]
#     d2 = directed_hausdorff(y_pred_points, y_true_points)[0]
#     return max(d1, d2)

# # Function to compute Precision, Recall, and F1 Score
# def compute_classification_metrics(y_true, y_pred):
#     y_true_flat = y_true.flatten()
#     y_pred_flat = y_pred.flatten()
    
#     precision = precision_score(y_true_flat, y_pred_flat, average="macro", zero_division=0)
#     recall = recall_score(y_true_flat, y_pred_flat, average="macro", zero_division=0)
#     f1 = f1_score(y_true_flat, y_pred_flat, average="macro", zero_division=0)
    
#     return precision, recall, f1

# # Function to compute Accuracy
# def pixel_accuracy(y_true, y_pred):
#     return np.mean(y_true == y_pred)

# # Make sure model predictions are working
# print("Making predictions...")
# Y_pred = model.predict(X_val)
# print("Predictions completed.")

# # Convert predictions to class indices
# Y_pred = np.argmax(Y_pred, axis=-1)
# Y_true = np.argmax(Y_val_one_hot, axis=-1)

# num_classes = 6  # Update this based on your dataset
# results = []

# print("Computing metrics for each class...")

# # Loop through each class and compute metrics
# for class_idx in range(num_classes):
#     print(f"Processing Class {class_idx}...")

#     y_true_class = (Y_true == class_idx).astype(np.uint8)
#     y_pred_class = (Y_pred == class_idx).astype(np.uint8)

#     dice = dice_score(y_true_class, y_pred_class)
#     iou = iou_score(y_true_class, y_pred_class)
#     hausdorff = hausdorff_distance(y_true_class, y_pred_class)
#     precision, recall, f1 = compute_classification_metrics(y_true_class, y_pred_class)
#     accuracy = pixel_accuracy(y_true_class, y_pred_class)

#     results.append([class_idx, dice, iou, hausdorff, precision, recall, f1, accuracy])

# # Convert to DataFrame for table formatting
# df_results = pd.DataFrame(results, columns=["Class", "Dice Score", "IoU", "Hausdorff Distance", 
#                                             "Precision", "Recall", "F1 Score", "Accuracy"])

# # Replace NaN values if any (Hausdorff may return NaN)
# df_results = df_results.fillna(0)

# # Display the table
# display(df_results)


In [ ]:
#df_results.to_csv("/kaggle/working/evaluation_output.csv", index=False)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Select an image from the validation set
manual_idx = 6
img = X_val[manual_idx]  # Original OCT scan
true_mask = Y_val[manual_idx]  # Ground truth segmentation mask

# Debugging: Print the shape of true_mask
print("Shape of true_mask before processing:", true_mask.shape)

# Convert the true mask to RGB for visualization
def label_to_rgb(label_mask):
    # Ensure label_mask is 2D by squeezing and indexing if necessary
    label_mask = np.squeeze(label_mask)  # Remove single-dimensional entries
    
    # Debugging: Print the shape after squeezing
    print("Shape of label_mask after squeezing:", label_mask.shape)
    
    # Handle cases where label_mask still has more than 2 dimensions
    if len(label_mask.shape) > 2:
        # If there are multiple channels, take the first channel
        label_mask = label_mask[:, :, 0]
        print("Shape of label_mask after selecting first channel:", label_mask.shape)
    
    # Now unpack the shape
    H, W = label_mask.shape
    rgb_mask = np.zeros((H, W, 3), dtype=np.uint8)
    
    # Define a color map for different labels
    COLOR_MAP = {
    0: (255, 0, 0),  # Red
    1: (255, 192, 203),  # Pink
    2: (0, 0, 255),  # Blue
    3: (0, 255, 0),  # Green
    4: (255, 255, 0),  # Yellow
    5: (0, 0, 0)  # Black (background)
}
    
    for class_idx, color in COLOR_MAP.items():
        rgb_mask[label_mask == class_idx] = color
    
    return rgb_mask

# Ensure true_mask is 2D before passing it to label_to_rgb
true_mask_rgb = label_to_rgb(true_mask)

# Function to compute saliency maps
class SaliencyMap:
    def __init__(self, model):
        self.model = model

    def __call__(self, input_tensor, class_idx=None):
        # Ensure the input tensor has batch dimension
        input_tensor = tf.convert_to_tensor(input_tensor[np.newaxis, ...], dtype=tf.float32)
        
        with tf.GradientTape() as tape:
            tape.watch(input_tensor)
            predictions = self.model(input_tensor)  # Shape: (1, H, W, num_classes)
            
            # If no class index is specified, use the predicted class for the first pixel
            if class_idx is None:
                class_idx = tf.argmax(predictions[0, 0, 0])  # Get class of first pixel
            
            # Compute loss as sum of probabilities for the target class across all pixels
            loss = tf.reduce_sum(predictions[:, :, :, class_idx])

        # Compute gradients of the loss w.r.t. the input image
        grads = tape.gradient(loss, input_tensor)[0]  # Remove batch dimension
        
        # Compute the absolute value of gradients and normalize
        saliency = tf.reduce_max(tf.abs(grads), axis=-1)  # Max across RGB channels
        saliency = (saliency - tf.reduce_min(saliency)) / (tf.reduce_max(saliency) - tf.reduce_min(saliency) + 1e-6)
        return saliency.numpy()

# Initialize Saliency Map
saliency_map = SaliencyMap(model)

# Generate the saliency map for a specific class (e.g., class 3)
saliency = saliency_map(img, class_idx=3)

# Predict the segmentation mask using the trained model
predicted = model.predict(np.expand_dims(img, axis=0))
predicted_mask = np.argmax(predicted[0], axis=-1)  # Shape: (192, 288)
predicted_mask_rgb = label_to_rgb(predicted_mask)  # Convert predicted mask to RGB

# Plot the results
plt.figure(figsize=(18, 6))

# Panel 1: Saliency map overlaid on the original image
plt.subplot(1, 3, 1)
plt.imshow(img)
plt.imshow(saliency, cmap="jet", alpha=0.5)
plt.title("Original Image with Saliency Map", fontsize=19)
plt.axis("off")

# Panel 2: Saliency map overlaid on the ground truth mask
plt.subplot(1, 3, 2)
plt.imshow(true_mask_rgb)
plt.imshow(saliency, cmap="jet", alpha=0.5)
plt.title("Ground Truth Mask with Saliency Map", fontsize=19)
plt.axis("off")

# Panel 3: Saliency map overlaid on the predicted mask
plt.subplot(1, 3, 3)
plt.imshow(predicted_mask_rgb)
plt.imshow(saliency, cmap="jet", alpha=0.5)
plt.title("Predicted Mask with Saliency Map", fontsize=19)
plt.axis("off")

# Save the figure
plt.tight_layout()
plt.savefig("/kaggle/working/saliency_visualization.png")  # Save as PNG
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Install the grad-cam library if not already installed
!pip install grad-cam

# Select an image from the validation set
manual_idx = 20
img = X_val[manual_idx]  # Original OCT scan
true_mask = Y_val[manual_idx]  # Ground truth segmentation mask

# Debugging: Print the initial shape of true_mask
print("Initial shape of true_mask:", true_mask.shape)

# Convert the true mask to RGB for visualization
def label_to_rgb(label_mask):
    # Ensure label_mask is 2D by squeezing extra dimensions
    label_mask = np.squeeze(label_mask)  # Remove single-dimensional entries
    
    # Debugging: Print the shape after squeezing
    print("Shape of label_mask after squeezing:", label_mask.shape)
    
    # Handle cases where label_mask still has more than 2 dimensions
    if len(label_mask.shape) > 2:
        # If there are multiple channels, take the first channel
        label_mask = label_mask[:, :, 0]
        print("Shape of label_mask after selecting first channel:", label_mask.shape)
    
    # Now unpack the shape
    H, W = label_mask.shape
    rgb_mask = np.zeros((H, W, 3), dtype=np.uint8)
    
    # Define the global COLOR_MAP
    COLOR_MAP = {
        0: (255, 0, 0),      # Red
        1: (255, 192, 203),  # Pink
        2: (0, 0, 255),      # Blue
        3: (0, 255, 0),      # Green
        4: (255, 255, 0),    # Yellow
        5: (0, 0, 0)         # Black (background)
    }
    
    # Assign colors based on the mask values
    for class_idx, color in COLOR_MAP.items():
        rgb_mask[label_mask == class_idx] = color
    
    return rgb_mask

# Ensure true_mask is 2D before passing it to label_to_rgb
true_mask_rgb = label_to_rgb(true_mask)

# Apply Grad-CAM to the student model
layer_name = "conv2d_1"  # Replace with the name of a convolutional layer in your model

# Wrap the TensorFlow/Keras model in a GradCAM-compatible class
class KerasGradCAM:
    def __init__(self, model, layer_name):
        self.model = model
        self.layer_name = layer_name

    def __call__(self, input_tensor, class_idx=None):
        # Ensure the input tensor has batch dimension
        input_tensor = np.expand_dims(input_tensor, axis=0)
        
        # Create a sub-model that outputs both the target layer and the model predictions
        grad_model = tf.keras.models.Model(
            inputs=self.model.inputs,
            outputs=[self.model.get_layer(self.layer_name).output, self.model.output]
        )

        # Use GradientTape to compute gradients
        with tf.GradientTape() as tape:
            conv_outputs, predictions = grad_model(input_tensor)
            
            # If no class index is specified, use the predicted class for the first pixel
            if class_idx is None:
                class_idx = tf.argmax(predictions[0, 0, 0])  # Get class of first pixel
            
            # Compute loss as sum of probabilities for the target class across all pixels
            loss = tf.reduce_sum(predictions[:, :, :, class_idx])

        # Compute gradients of the loss w.r.t. the target layer output
        grads = tape.gradient(loss, conv_outputs)[0]
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1))  # Global average pooling

        # Weight the feature maps by the pooled gradients
        conv_outputs = conv_outputs[0]
        heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
        heatmap = np.maximum(heatmap, 0)  # ReLU activation
        heatmap /= np.max(heatmap) if np.max(heatmap) > 0 else 1  # Normalize
        
        # Return the heatmap (already a NumPy array)
        return heatmap  # Removed .numpy() here

# Initialize Grad-CAM
grad_cam = KerasGradCAM(model, layer_name="conv2d_1")

# Generate the heatmap
heatmap = grad_cam(img, class_idx=3)  # Specify class_idx if needed (e.g., class 3)

# Resize the heatmap to match the dimensions of the input image and ground truth mask
heatmap_resized = tf.image.resize(heatmap[np.newaxis, ..., np.newaxis], 
                                  img.shape[:2],
                                  method=tf.image.ResizeMethod.BILINEAR).numpy()[0, :, :, 0]

# Predict the segmentation mask using the trained model (distiller)
predicted = model.predict(np.expand_dims(img, axis=0))
predicted_mask = np.argmax(predicted[0], axis=-1)  # Shape: (192, 288)
predicted_mask_rgb = label_to_rgb(predicted_mask)  # Convert predicted mask to RGB

# Plot the results
plt.figure(figsize=(18, 6))

# Panel 1: Heatmap overlaid on the original image
plt.subplot(1, 3, 1)
plt.imshow(img)
plt.title("Original OCT Scan", fontsize=19)
plt.axis("off")

# Panel 2: Heatmap overlaid on the original image
plt.subplot(1, 3, 2)
plt.imshow(img)
plt.imshow(heatmap_resized, cmap="jet", alpha=0.5)
plt.title("Original Image with Grad-CAM Heatmap", fontsize=19)
plt.axis("off")

# Panel 3: Saliency map overlaid on the original image
plt.subplot(1, 3, 3)
plt.imshow(img)
plt.imshow(saliency, cmap="jet", alpha=0.5)
plt.title("Original Image with Saliency Map", fontsize=19)
plt.axis("off")

# Save the figure
plt.tight_layout()
plt.savefig("/kaggle/working/gradcam1_visualization.png")  # Save as PNG
plt.show()